In [2]:
# Importing all the essential libraries
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import re
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [4]:
#Define a list of movies numbers which are at start of the page
li=np.arange(1,5001,50)

Extracting data of movies from 2010 to 2019 sorted gross collections in descending order. Total of 5000 movies were extracted

In [58]:
# Create an empty list of variables which would be scrapped
name=[]
year=[]
rating=[]
length=[]
votes=[]
certificate=[]
metascore=[]
genre=[]
hero=[]
director=[]
gross=[]

for i in li:
    url='https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series,short&release_date=2010-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start={}&ref_=adv_nxt'.format(i)
    page=requests.get(url)
    content=page.text
    soup=BeautifulSoup(content)
    parent_div=soup.find_all('div',{'class':'lister-item-content'})
    
    
    for j in parent_div:
        
        name.append(j.h3.a.text)
        
        if j.h3.span.text:
            year.append(j.h3.span.text)
        else:
            year.append("NA")
      
        if j.strong:
            rating.append(j.strong.text)
        else:
            rating.append("NA")
       
        if j.find_all('span',{'name':'nv'}):
            votes.append(j.find_all('span',{'name':'nv'})[0]["data-value"])
        else:
            votes.append("NA")
        
        if j.p.find('span',{'class':'genre'}):
            genre.append(j.p.find('span',{'class':'genre'}).text)
        else:
            genre.append("NA")
        
        if j.p.span:
            certificate.append(j.p.span.text)
        else:
            certificate.append("NA")
            
        if j.p.find("span",{'class':'run-time'}):
            length.append(j.p.find('span',{'class':'runtime'}).text)
        else:
            length.append("NA")
            
        
        if j.find('div', class_ = 'ratings-metascore') is not None: 
            metascore.append(j.find('div', class_ = 'ratings-metascore').span.text)
        else:
            metascore.append("NA")
            
            
        if j.find_all('p')[2].text.find("Director")>0:
            director.append(j.find_all('p')[2].text[j.find_all('p')[2].text.find("Director"):j.find_all('p')[2].text.find("|")])
        else:
            director.append("NA")
       
    
        if j.find_all('p')[2].text.find("Stars")>0:
            director.append(j.find_all('p')[2].text[j.find_all('p')[2].text.find("Stars"):])
        else:
            director.append("NA")
                
                
        if len(j.find_all('span',{'name':'nv'}))==2:
            gross.append(j.find_all('span',{'name':'nv'})[1]["data-value"])
        else:
            gross.append("NA")

In [ ]:
# Create a dataset to store raw data and name it as raw
raw=pd.DataFrame(list(zip(name, year, rating, length, votes, certificate, metascore, genre, hero, director, gross)), columns=["Name", "Year", "Rating", "Length","Votes","Certificate","Metascore","Genre","Hero","Director","Gross"])

In [172]:
raw.head()

,Name,Year,Rating,Length,Votes,Certificate,Metascore,Genre,Hero,Director,Gross
0,Star Wars: Episode VII - The Force Awakens,(2015),7.9,138 min,868933,U,80,"\nAction, Adventure, Sci-Fi","Stars:\nDaisy Ridley, \nJohn Boyega, \nOscar I...",Director:\nJ.J. Abrams\n,"936,662,225"
1,Avengers: Endgame,(2019),8.4,181 min,860645,UA,78,"\nAction, Adventure, Drama","Stars:\nRobert Downey Jr., \nChris Evans, \nMa...","Directors:\nAnthony Russo, \nJoe Russo\n","858,373,000"
2,Black Panther,(2018),7.3,134 min,642941,UA,88,"\nAction, Adventure, Sci-Fi","Stars:\nChadwick Boseman, \nMichael B. Jordan,...",Director:\nRyan Coogler\n,"700,059,566"
3,Avengers: Infinity War,(2018),8.4,149 min,868233,UA,68,"\nAction, Adventure, Sci-Fi","Stars:\nRobert Downey Jr., \nChris Hemsworth, ...","Directors:\nAnthony Russo, \nJoe Russo\n","678,815,482"
4,Jurassic World,(2015),7.0,124 min,585609,UA,59,"\nAction, Adventure, Sci-Fi","Stars:\nChris Pratt, \nBryce Dallas Howard, \n...",Director:\nColin Trevorrow\n,"652,270,625"


In [180]:
#raw.to_csv("Raw_2011-2019.csv", index=False)

# Cleaning data

Extracting only required number from entry and converting it into integer datatype for variables "Year", "Length", "Gross"

In [18]:
def integer(x):
    if x=="NA":
        return None
    else:
        num=''.join([i for i in x if i.isdigit()])
        return int(num)

In [177]:
raw["Year"]=raw["Year"].apply(lambda x:integer(x))

In [178]:
raw["Length"]=raw["Length"].apply(lambda x: integer(x))

In [179]:
raw["Gross"]=raw["Gross"].apply(lambda x: integer(x))

In [181]:
raw.dtypes

Name            object
Year             int64
Rating          object
Length         float64
Votes           object
Certificate     object
Metascore       object
Genre           object
Hero            object
Director        object
Gross          float64
dtype: object

In [4]:
raw.head()

,Name,Year,Rating,Length,Votes,Certificate,Metascore,Genre,Hero,Director,Gross
0,Star Wars: Episode VII - The Force Awakens,2015,7.9,138.0,868933,U,80.0,"\nAction, Adventure, Sci-Fi","Stars:\nDaisy Ridley, \nJohn Boyega, \nOscar I...",Director:\nJ.J. Abrams\n,936662225.0
1,Avengers: Endgame,2019,8.4,181.0,860645,UA,78.0,"\nAction, Adventure, Drama","Stars:\nRobert Downey Jr., \nChris Evans, \nMa...","Directors:\nAnthony Russo, \nJoe Russo\n",858373000.0
2,Black Panther,2018,7.3,134.0,642941,UA,88.0,"\nAction, Adventure, Sci-Fi","Stars:\nChadwick Boseman, \nMichael B. Jordan,...",Director:\nRyan Coogler\n,700059566.0
3,Avengers: Infinity War,2018,8.4,149.0,868233,UA,68.0,"\nAction, Adventure, Sci-Fi","Stars:\nRobert Downey Jr., \nChris Hemsworth, ...","Directors:\nAnthony Russo, \nJoe Russo\n",678815482.0
4,Jurassic World,2015,7.0,124.0,585609,UA,59.0,"\nAction, Adventure, Sci-Fi","Stars:\nChris Pratt, \nBryce Dallas Howard, \n...",Director:\nColin Trevorrow\n,652270625.0


Removing "\n", "Stars", "Director", ":" strings from variables "Hero", "Director"

In [5]:
def remsdn(x):
    if pd.notna(x):
        for i in ["Stars","Director","\n",":"]:
            x=x.replace(i,'')
        return str(x)
    else:
        return x

In [9]:
raw["Hero"]=raw["Hero"].apply(lambda x: remsdn(x))

In [11]:
raw["Director"]=raw["Director"].apply(lambda x:remsdn(x))

Removing "\n", left spaces and right spaces from variables "Genre"

In [14]:
def strip(x):
    if pd.notna(x):
        return x.strip().lstrip().rstrip()
    else:
        return None

In [16]:
raw["Genre"]=raw["Genre"].apply(lambda x:strip(x))

In [20]:
def integerr(x):
    if pd.notna(x):
        return int("".join(re.findall(r'\d',x)))
    else:
        return x

In [23]:
raw["Votes"]=raw["Votes"].apply(lambda x:integerr(x))

In [24]:
raw.head()

,Name,Year,Rating,Length,Votes,Certificate,Metascore,Genre,Hero,Director,Gross
0,Star Wars: Episode VII - The Force Awakens,2015,7.9,138.0,868933.0,U,80.0,"Action, Adventure, Sci-Fi","Daisy Ridley, John Boyega, Oscar Isaac, Domhna...",J.J. Abrams,936662225.0
1,Avengers: Endgame,2019,8.4,181.0,860645.0,UA,78.0,"Action, Adventure, Drama","Robert Downey Jr., Chris Evans, Mark Ruffalo, ...","sAnthony Russo, Joe Russo",858373000.0
2,Black Panther,2018,7.3,134.0,642941.0,UA,88.0,"Action, Adventure, Sci-Fi","Chadwick Boseman, Michael B. Jordan, Lupita Ny...",Ryan Coogler,700059566.0
3,Avengers: Infinity War,2018,8.4,149.0,868233.0,UA,68.0,"Action, Adventure, Sci-Fi","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...","sAnthony Russo, Joe Russo",678815482.0
4,Jurassic World,2015,7.0,124.0,585609.0,UA,59.0,"Action, Adventure, Sci-Fi","Chris Pratt, Bryce Dallas Howard, Ty Simpkins,...",Colin Trevorrow,652270625.0


The dataset is ready for analysis